# Store kitti sequence

Create a sequence

In [149]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
os.environ['SEQUENCE_CONFIDENCE'] = '1.8'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [176]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_0'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_1'

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 18.3 ms, sys: 4.01 ms, total: 22.3 ms
Wall time: 21.6 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [177]:
seq = Sequence()
for f in ProgressBar()(frame[:15]):
    seq.add_keyframe(f)

N/A% (0 of 15) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

set_previous_keyframe 18272


  6% (1 of 15) |#                         | Elapsed Time: 0:00:03 ETA:  0:00:44

get_world_pose first  18272
set_previous_keyframe 18273


 13% (2 of 15) |###                       | Elapsed Time: 0:00:07 ETA:  0:00:45

...

get_world_pose  18284 [[ 0.99697398 -0.01682406  0.07589354 -1.37436329]
 [ 0.01768352  0.99978673 -0.01066675  0.38135783]
 [-0.07569789  0.01197654  0.99705887 -8.45033932]
 [ 0.          0.          0.          1.        ]]
set_previous_keyframe 18285


 93% (14 of 15) |#######################  | Elapsed Time: 0:00:58 ETA:  0:00:04

get_world_pose  18285 [[ 0.99676065 -0.01360441  0.07926617 -1.46798489]
 [ 0.01447574  0.99984084 -0.01042814  0.3994181 ]
 [-0.07911168  0.01154179  0.99679894 -9.30306928]
 [ 0.          0.          0.          1.        ]]
set_previous_keyframe 18286


100% (15 of 15) |#########################| Elapsed Time: 0:01:01 Time: 0:01:01


get_world_pose  18286 [[  0.99652214  -0.01378244   0.08218066  -1.5521411 ]
 [  0.01471414   0.99983403  -0.01074229   0.40684799]
 [ -0.08201897   0.01191415   0.99655955 -10.15504945]
 [  0.           0.           0.           1.        ]]


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [178]:
print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.get_world_pose().flatten() for kf in keyframes ] ])

Keyframes count
 15 



In [179]:
#for f in seq.keyframes:
#    print(f.get_world_pose())

In [180]:
cv_keyframes = get_covisible_keyframes(seq.keyframes[14])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

#[f.frameid for f in cv_keyframes]
cv_keyframes_np[:, 2:]

array([[  9.99596359e-01,   3.68446009e-05,   2.84098242e-02,
         -2.24975907e-01,  -2.35971980e-04,   9.99975431e-01,
          7.00578206e-03,  -2.07130527e-02,  -2.84088681e-02,
         -7.00965816e-03,   9.99571809e-01,  -3.79196672e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.99378566e-01,  -1.17268018e-03,   3.52293431e-02,
         -2.96417675e-01,   9.51869278e-04,   9.99979803e-01,
          6.28393044e-03,   1.83013817e-02,  -3.52360006e-02,
         -6.24649166e-03,   9.99359498e-01,  -4.61303414e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.99093138e-01,  -3.81515110e-03,   4.24069143e-02,
         -3.75784812e-01,   3.59376723e-03,   9.99979521e-01,
          5.29547587e-03,   5.88981527e-02,  -4.24262489e-02,
         -5.13827303e-03,   9.99086388e-01,  -5.45246456e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.0000

In [181]:
cv_keyframes_BA[:,2:]

array([[  9.99596359e-01,   3.68446009e-05,   2.84098242e-02,
         -2.24975907e-01,  -2.35971980e-04,   9.99975431e-01,
          7.00578206e-03,  -2.07130527e-02,  -2.84088681e-02,
         -7.00965816e-03,   9.99571809e-01,  -3.79196672e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.99378566e-01,  -1.17268018e-03,   3.52293431e-02,
         -2.96417675e-01,   9.51869278e-04,   9.99979803e-01,
          6.28393044e-03,   1.83013817e-02,  -3.52360006e-02,
         -6.24649166e-03,   9.99359498e-01,  -4.61303414e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.98877076e-01,  -3.83294772e-03,   4.72217788e-02,
         -4.02061877e-01,   3.53758213e-03,   9.99973665e-01,
          6.33684490e-03,   5.32304501e-02,  -4.72448240e-02,
         -6.16267818e-03,   9.98864329e-01,  -5.45026176e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.0000

In [182]:

cv_keyframes = get_covisible_keyframes(seq.keyframes[14])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

# print('mappoints count  \n', (len(mappoints[:]), '\n') )
# print('mappoints \n', (mappoints[:]), '\n')
# print('links count  \n', (len(links[:]), '\n') )
# print('links \n', (links[:]), '\n')

# print('Keyframes \n', (cv_keyframes), '\n')
# print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
# print('mappoints len \n', len(mappoints_np), '\n')
# print('Fixed Keyframes \n',f_keyframes[:], '\n')

# print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
# print(len(cv_keyframes_BA), '\n')

outliers =  urbg2o.outliersForLocalBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

# print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
# print(len(cv_keyframes_BA), '\n')

In [195]:
from sortedcontainers import SortedSet

inliers = { (int(pointid), int(frameid)) for frameid, pointid, chi2 in outliers if chi2 < 100000000 }
keptpoints = SortedSet({ l[0] for l in inliers })
keptframes = SortedSet({ l[1] for l in inliers })
#m_keptpoints = { p:i for i, p in enumerate(keptpoints) }
#m_keptframes = { p:i for i, p in enumerate(keptframes) }
cv_keyframes1 = [ f for f in cv_keyframes if f.keyframeid in keptframes]
fixed_keyframes1 = [ f for f in f_keyframes if f.keyframeid in keptframes]
mappoints1 = [ f for f in mappoints if int(f[0]) in keptpoints]
links1 = [ (l[0], l[1], l[2], l[3]) for l in links if (int(l[0]), int(l[1])) in inliers ]

In [196]:
len(mappoints1), len(links1)

(1072, 2785)

In [197]:
len(inliers) / len(links)

0.8112438100786484

In [198]:
cv_keyframes1_np = keyframes_to_np(cv_keyframes1)
f_keyframes1_np = keyframes_to_np(fixed_keyframes1)
links1_np = np.array(links1, dtype=np.float64, order='f')

f_keyframes1 = np.asfortranarray(f_keyframes1_np)
cv_keyframes1_BA = np.asfortranarray(cv_keyframes1_np)
mappoints1 = np.asfortranarray(mappoints1)
links1 = np.array(links1_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes1_BA, f_keyframes1, mappoints1, links1)



In [199]:
cv_keyframes1_BA[:,2:]

array([[  9.99596359e-01,   3.68446009e-05,   2.84098242e-02,
         -2.24975907e-01,  -2.35971980e-04,   9.99975431e-01,
          7.00578206e-03,  -2.07130527e-02,  -2.84088681e-02,
         -7.00965816e-03,   9.99571809e-01,  -3.79196672e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.99378566e-01,  -1.17268018e-03,   3.52293431e-02,
         -2.96417675e-01,   9.51869278e-04,   9.99979803e-01,
          6.28393044e-03,   1.83013817e-02,  -3.52360006e-02,
         -6.24649166e-03,   9.99359498e-01,  -4.61303414e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.99770517e-01,  -2.03393323e-03,   2.13255076e-02,
          7.69435895e-02,   1.82376717e-03,   9.99949628e-01,
          9.86997070e-03,  -9.04994899e-02,  -2.13445083e-02,
         -9.82881295e-03,   9.99723865e-01,  -4.59700501e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.0000

In [13]:

cv_keyframes = { seq.keyframes[i] for i in range(0,7,2) }
mappoints = get_mappoints(cv_keyframes)
f_keyframes = { }

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

# print('mappoints count  \n', (len(mappoints[:]), '\n') )
# print('mappoints \n', (mappoints[:]), '\n')
# print('links count  \n', (len(links[:]), '\n') )
# print('links \n', (links[:]), '\n')

# print('Keyframes \n', (cv_keyframes), '\n')
# print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
# print('mappoints len \n', len(mappoints_np), '\n')
# print('Fixed Keyframes \n',f_keyframes[:], '\n')

# print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
# print(len(cv_keyframes_BA), '\n')

outliers =  urbg2o.outliersForLocalBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

# print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
# print(len(cv_keyframes_BA), '\n')

In [14]:
from sortedcontainers import SortedSet

inliers = { (int(pointid), int(frameid)) for frameid, pointid, chi2 in outliers if chi2 < 10 }
keptpoints = SortedSet({ l[0] for l in inliers })
keptframes = SortedSet({ l[1] for l in inliers })
m_keptpoints = { p:i for i, p in enumerate(keptpoints) }
m_keptframes = { p:i for i, p in enumerate(keptframes) }
cv_keyframes1 = [ f for i, f in enumerate(cv_keyframes) if i in keptframes]
fixed_keyframes1 = [ f for i, f in enumerate(f_keyframes) if i + len(cv_keyframes) in keptframes]
mappoints1 = [ f for i, f in enumerate(mappoints) if i in keptpoints]
links1 = [ (m_keptpoints[l[0]], m_keptframes[l[1]], l[2], l[3]) for l in links if (int(l[0]), int(l[1])) in inliers ]

In [15]:
len(inliers) / len(links)

0.2057335581787521

In [16]:
cv_keyframes1_np = keyframes_to_np(cv_keyframes1)
f_keyframes1_np = keyframes_to_np(fixed_keyframes1)
links1_np = np.array(links1, dtype=np.float64, order='f')

f_keyframes1 = np.asfortranarray(f_keyframes1_np)
cv_keyframes1_BA = np.asfortranarray(cv_keyframes1_np)
mappoints1 = np.asfortranarray(mappoints1)
links1 = np.array(links1_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes1_BA, f_keyframes1, mappoints1, links1)



In [17]:
cv_keyframes1_BA[:, 2:]

array([[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.96785922e-01,  -3.15546374e-02,   7.36351131e-02,
         -1.56032522e+00,   3.16368583e-02,   9.99499428e-01,
          4.98007851e-05,   3.80801591e-01,  -7.35998249e-02,
          2.27994292e-03,   9.97285249e-01,  -1.85248486e+01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])